In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
import torch 
from torch import nn
from torch.nn import functional as F
from torch.utils.data import Dataset, DataLoader, random_split
from string import printable
import numpy as np


In [0]:
batch_size = 8
with open('/content/drive/My Drive/Colab Notebooks/Practice/CharacterGeneration/file.txt', 'r') as f:
    text = f.read()

text_size = len(text)
split_ratio = 0.9
train_text = text[:int(split_ratio * text_size)]
test_text = text[int(split_ratio * text_size):]

In [0]:
class LanguageModelDatset(Dataset):
    def __init__(self, text, sequence_length=100):
        super().__init__()
        self.device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
        self.sequence_length = 100
        self.text = text
        self.itos = list(printable)
        self.stoi = {char:int_ for int_, char in enumerate(self.itos)}
        self.text_size = len(self.text)
        
    def one_hot_encoder(self, sequence):
        size = len(self.stoi)

        encoded = []
        for int_ in sequence:
            temp = torch.zeros(size)
            temp[int_] = 1
            encoded.append(temp)

        for i in range(self.sequence_length - len(sequence) - 1):
            temp = torch.zeros(size)
            encoded.append(temp)

        return torch.stack(encoded)
        
    def __len__(self):
        return self.text_size // self.sequence_length
    
    def __getitem__(self, idx):
        sequence = self.text[idx * self.sequence_length:(idx + 1) * self.sequence_length]
        sequence = [self.stoi[x] for x in sequence]
        x = sequence[:-1]
        y = sequence[1:]
        x = self.one_hot_encoder(x)
        
        return x, torch.tensor(y)
        

In [0]:
max_test_loss = float('inf')
train_dataset = LanguageModelDatset(train_text)
test_dataset = LanguageModelDatset(test_text)
train_dataloader = DataLoader(train_dataset, batch_size=batch_size)
test_dataloader = DataLoader(test_dataset, batch_size=batch_size)

In [6]:
print(train_dataset[0][0].shape)
print(train_dataset[0][1].shape)

torch.Size([99, 100])
torch.Size([99])


In [0]:
class CharLSTM(nn.Module):
    def __init__(self, hidden_size=256, n_hidden_layers=3, dropout_prob=0.5):
        super().__init__()
        self.legal_chars = printable
        self.hidden_size = hidden_size
        self.n_hidden_layers = n_hidden_layers
        self.device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
        self.num_layers = n_hidden_layers
        
        self.lstm_layer = nn.LSTM(
            input_size=len(self.legal_chars), 
            hidden_size=hidden_size, 
            num_layers=n_hidden_layers, 
            dropout=dropout_prob, 
            batch_first=True
        )
        self.dropout_layer = nn.Dropout(dropout_prob)
        self.fc = nn.Linear(hidden_size, len(self.legal_chars))
        self.to(self.device)

    def forward(self, x):
        self.hidden = self.hidden.detach()
        self.cell = self.cell.detach()
        x = x.to(self.device)
        lstm_output, (self.hidden, self.cell) = self.lstm_layer(x, (self.hidden, self.cell))
        output = self.dropout_layer(lstm_output)
        output = output.reshape(-1, self.hidden_size)
        return self.fc(output)

    def init_hidden(self, batch_size):
        # hidden state and cell state
        self.hidden, self.cell = [torch.zeros(self.num_layers, batch_size, self.hidden_size).to(self.device), 
                torch.zeros(self.num_layers, batch_size, self.hidden_size).to(self.device)]

In [8]:
model = CharLSTM()
model

CharLSTM(
  (lstm_layer): LSTM(100, 256, num_layers=3, batch_first=True, dropout=0.5)
  (dropout_layer): Dropout(p=0.5, inplace=False)
  (fc): Linear(in_features=256, out_features=100, bias=True)
)

In [0]:
lr = 1e-3
num_epochs = 10
sequence_length = 99
clip = 5
model_path = '/content/drive/My Drive/Colab Notebooks/Practice/CharacterGeneration/model.pth'

In [11]:
import os


def save_weights(model, path):
    torch.save(model.state_dict(), path)
    
def load_weights(model, path, test=False):
    model.load_state_dict(torch.load(path))
    if test:
        model.eval()
    else:
        model.train()

if os.path.exists(model_path):
    print('Existing model found!')
    load_weights(model, model_path)
    
else:
    print('No existing model.')

No existing model.


In [20]:
opt = torch.optim.Adam(model.parameters(), lr=lr)
criterion = nn.CrossEntropyLoss()
print_every = 500

for epoch in range(num_epochs):
    model.train()
    model.init_hidden(batch_size)
    counter = 0
    for x, y in train_dataloader:
        if x.shape[0] != batch_size:
            continue
        counter += 1
        opt.zero_grad()
        output = model(x)
        loss = criterion(output, y.long().view(-1,).to(model.device))
        loss.backward()
        nn.utils.clip_grad_norm_(model.parameters(), clip)
        opt.step()

        if counter % print_every == 0:
            print("Epoch: {}/{}...".format(epoch+1, num_epochs),
                  "Step: {}...".format(counter),
                  "Loss: {:.4f}...".format(loss.item()))
            
    with torch.no_grad():
        model.eval()
        test_loss = 0
        total = len(test_dataset)
        for x, y in test_dataloader:
            if x.shape[0] != batch_size:
                continue
            opt.zero_grad()
            output = model(x)
            loss = criterion(output, y.long().view(-1,).to(model.device))
            test_loss += loss.item()

        print(f'=' * 8 + f'\nTest loss: {test_loss/total}\n' + '=' * 8)
        if test_loss < max_test_loss:
            max_test_loss = test_loss
            save_weights(model, model_path)


Epoch: 1/10... Step: 500... Loss: 1.3991...
Epoch: 1/10... Step: 1000... Loss: 1.2480...
Epoch: 1/10... Step: 1500... Loss: 1.5717...
Epoch: 1/10... Step: 2000... Loss: 1.3722...
Epoch: 1/10... Step: 2500... Loss: 1.5337...
Epoch: 1/10... Step: 3000... Loss: 1.4409...
Epoch: 1/10... Step: 3500... Loss: 1.3209...
Epoch: 1/10... Step: 4000... Loss: 1.4717...
Epoch: 1/10... Step: 4500... Loss: 1.4736...
Epoch: 1/10... Step: 5000... Loss: 1.4376...
Epoch: 1/10... Step: 5500... Loss: 1.4301...
Epoch: 1/10... Step: 6000... Loss: 1.4231...
Epoch: 1/10... Step: 6500... Loss: 1.3491...
Epoch: 1/10... Step: 7000... Loss: 1.5761...
Test loss: 0.16586940247955417
Epoch: 2/10... Step: 500... Loss: 1.3886...
Epoch: 2/10... Step: 1000... Loss: 1.2528...
Epoch: 2/10... Step: 1500... Loss: 1.5512...
Epoch: 2/10... Step: 2000... Loss: 1.3810...
Epoch: 2/10... Step: 2500... Loss: 1.6003...
Epoch: 2/10... Step: 3000... Loss: 1.4184...
Epoch: 2/10... Step: 3500... Loss: 1.3155...
Epoch: 2/10... Step: 4000.

In [0]:
def predict_next_char(model, char, h=None):
    global test_dataset
    model.eval()
    char_vector = test_dataset.one_hot_encoder([char])[0]
    char_vector = char_vector.reshape(1, 1, *char_vector.shape)
    with torch.no_grad():
        model.hidden, model.cell = h
        output = model.forward(char_vector.to(model.device))
        h = (model.hidden, model.cell)
        probs = F.softmax(output, dim=1)

        # Sampling from a distribution to add randomness
        dist = torch.distributions.Categorical(probs)
        index = dist.sample().item()
        return index, h

def generate_chars(model, n_chars, prime='The'):
    global test_dataset
    model.eval()
    model.init_hidden(1)
    h = (model.hidden, model.cell)

    for char in [test_dataset.stoi[x] for x in prime]:
        _, h = predict_next_char(model, char, h)

    chars = []

    for i in range(n_chars):
        char, h = predict_next_char(model, char, h)
        chars.append(char)

    return prime + ''.join([test_dataset.itos[x] for x in chars])



In [22]:
print(generate_chars(model, n_chars=2000, prime='The'))

Thenoud. That may advents and then were what he had realized him to with it his word really he had hoped to gravely.

The Girls were to
directly down as Moscow "and many name in Frenchsiatures: still tried
to go
to the man and took peasants with recembling officer replied.

Without expectated her.

"To last after it of them; if he asked Doroth, do what a words she wrote out. Might Russia, and how he knew us to the princess. See Pierre close, I. When she saw the God, as he had khow. Napoleon's beauty-embings
he
opposite if and down to cur, beamed his spoze, remained gazed at his long dipfushius fur rash,
went in land. The
staff so endarked her stage.

There, evidently sat people, paper the Emperor respect and had a quarter appear mighter
and huers from the twill freed large. Count Molming which was loved to very offended kind collar out, and herself that like Borodino unattended to Prince Vasili. "I must be done not become meeting his will, so. Ing a restraint from her resist up to a sw